In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from IPython import display

# 語意即向量 (pytorch)

支援python 版本: 3.5以上  
支援pytorch版本 : 1.2以上

在深度學習中，所有計算的基礎都基於向量(張量)，而所謂複雜的問題，往往都是高維空間的計算問題。解決這些高維框間問題的最好辦法就是施以降維攻擊。文字從原本稀疏的onehot編碼，透過基於上下文猜中間的詞(cbow)的簡單任務，就能夠將文字的對應關係與語意從原本數百萬維(詞彙數)的複雜度，濃縮在200~300維之間，變成緊湊的語意特徵向量，這就是詞向量。飽含著濃縮的語意，因為維度被大幅降維，因此複雜的事務也會變簡單，簡單到只需要加減乘除外加上cosine similarity的計算就能解決的。這個實作是我們自然語言系列實做的第一個，我們就來看看這些語言特徵向量能夠理解語意到甚麼程度。

![Alt text](../images/word2vec.jpg)

In [2]:
import os
os.environ['TRIDENT_BACKEND'] = 'pytorch'

#!pip uninstall tridentx
#!pip install tridentx --upgrade
from trident import *
from typing import Optional,List,Tuple
import locale
import datetime
import tqdm

trident 0.7.4.1


Using Pytorch backend.
Image Data Format: channels_first.
Image Channel Order: rgb.
Pytorch version:1.10.1+cu113.
Automatic Mixed Precision Support:True.


Opencv version:4.5.4-dev.
Pillow version:8.4.0.


為了方便各位理解裡面每個計算的過程，以及順便讓各位看到如何在trident中擴充新的神經層，我把詞向量層的代碼附上。請注意，我這個詞向量當初是使用簡體語料訓練出來的，為了方便繁體中文使用者方便，我在裡面實現了基礎的繁簡轉換，但是不算完美，所以還請包涵(如果有人有足夠的繁體中文語料也請與我聯繫歐)。

In [11]:
from trident.models.pytorch_embedded import Word2Vec
w2v=Word2Vec.load()

model file is already existing, donnot need download again.
locale: zh_tw
vocabs_tw.txt: 402MB [00:01, 261MB/s]    
total loading time:0:00:24.032268


## 類比關係

首先我們當然是要來重現一下那個經典句子。

In [12]:
w2v.analogy('男人','國王','女人')

{ 王后: 0.6711671352386475, 路易十四: 0.6085006594657898, 國王隊: 0.6028344035148621, 薩克拉門托: 0.6001420021057129, 爵士: 0.599219560623169, 路易十五: 0.5940966606140137, 教皇: 0.5927847027778625, 拿破侖: 0.590887188911438, 萊恩: 0.5896217823028564, 喬治: 0.585080623626709 }

我們也要來試一下，被認為是深度學習偏見的經典案例的那一句。

![Alt text](../images/bias.png)

In [13]:
w2v.analogy('男人','工程師','女人')

{ 技術主管: 0.6507546305656433, 技術人員: 0.6456471085548401, 高級工程師: 0.6447331309318542, 設計員: 0.6362590193748474, 電氣工程師: 0.6353718638420105, 程序員: 0.6332074999809265, 技術員: 0.5936263799667358, 軟件開發: 0.5898590683937073, 項目經理: 0.5882405042648315, 研發部門: 0.5854535698890686 }

我們再試試其他的類比關係....

In [14]:
w2v.analogy('張惠妹','阿妹','周杰倫')

{ 周董: 0.8443138599395752, 杰倫: 0.7464919090270996, 哈林: 0.6755087971687317, 昆凌: 0.6727410554885864, 王力宏: 0.6535263061523438, 劉畊宏: 0.6101048588752747, 那姐: 0.5902319550514221, 吳宗憲: 0.5883661508560181, 林俊杰: 0.581734836101532, 方文山: 0.5801873803138733 }

In [15]:
w2v.analogy('雙子座',['花心','聰明'],'金牛座')

{ 厚道: 0.6142151951789856, 精明: 0.5781047940254211, 小氣: 0.5588236451148987, 福睿斯: 0.5313887000083923, 老實: 0.5267444252967834, 摳門: 0.5054975152015686, 專一: 0.488334596157074, 福特: 0.4873081147670746, 蒙迪歐: 0.4865235686302185, 翼虎: 0.4837643504142761 }

In [16]:
w2v.analogy('黃曉明','angelababy','胡歌')

{ 霍建華: 0.7969949841499329, 趙麗穎: 0.7508328557014465, 王凱: 0.745437741279602, 江疏影: 0.7421892881393433, 靳東: 0.7030094861984253, 唐嫣: 0.6812393665313721, 黃軒: 0.6749377250671387, 馬思純: 0.6685834527015686, 王子文: 0.6607146263122559, 楊洋: 0.6537554860115051 }

透過文字特徵的抽取，機器也能理解一些專業術語以及抽象關係

In [17]:
w2v.analogy('基金','贖回','期貨')

{ 平倉: 0.5166104435920715, 套保: 0.5061495304107666, 交割: 0.4786132574081421, 期貨市場: 0.47824665904045105, 期貨交易: 0.4673691689968109, 期市: 0.4625355899333954, 限倉: 0.45793458819389343, 期貨價格: 0.45300766825675964, 現貨: 0.4513823986053467, 交收: 0.448342889547348 }

In [18]:
w2v.analogy('鹵肉飯','米飯','牛肉面')

{ 面條: 0.8094527125358582, 饅頭: 0.7205346822738647, 白米飯: 0.7196974754333496, 湯面: 0.7152270674705505, 稀飯: 0.6803689002990723, 白飯: 0.6797652244567871, 面湯: 0.6796509623527527, 炒面: 0.6778761148452759, 玉米面: 0.6749952435493469, 清湯: 0.6691957116127014 }

## 列舉類似詞

列舉類似詞的原理很簡單，就是給幾個案例，然後我們去找尋跟這堆案例的距離總和最近的(cosine距離加總最大)。我們首先來看以下的案例。

In [19]:
w2v.get_enumerators('波蘭','捷克',n=20)

{ 捷克: 1.833512544631958, 波蘭: 1.8335121870040894, 匈牙利: 1.6807706356048584, 羅馬尼亞: 1.6704168319702148, 奧地利: 1.6387449502944946, 塞爾維亞: 1.601003885269165, 格魯吉亞: 1.5848054885864258, 斯洛伐克: 1.5822560787200928, 白俄羅斯: 1.5615065097808838, 烏克蘭: 1.5612664222717285, 保加利亞: 1.5587890148162842, 斯洛文尼亞: 1.5476024150848389, 克羅地亞: 1.5182490348815918, 拉脫維亞: 1.5003279447555542, 比利時: 1.4710290431976318, 愛沙尼亞: 1.4648041725158691, 立陶宛: 1.4634443521499634, 波黑: 1.4622795581817627, 亞美尼亞: 1.433415412902832, 葡萄牙: 1.4271326065063477 }

In [20]:
w2v.get_enumerators('美金','人民幣',n=20)

{ 人民幣: 1.5656778812408447, 美金: 1.5656774044036865, 美元: 1.5130164623260498, 韓元: 1.2852435111999512, 歐元: 1.273993730545044, 日元: 1.2584363222122192, 英鎊: 1.255664587020874, 盧比: 1.2380578517913818, 新台幣: 1.220036268234253, 比索: 1.1863694190979004, 台幣: 1.1631473302841187, 韓幣: 1.1575074195861816, 英磅: 1.1547527313232422, 澳幣: 1.1409335136413574, 盧布: 1.1278765201568604, 港元: 1.12125563621521, 澳元: 1.0988757610321045, 馬幣: 1.0820611715316772, 美圓: 1.078179121017456, 新幣: 1.0632843971252441 }

但是語意就是那麼複雜的東西，我們看看下面案例就知道複雜在哪了...

In [21]:
w2v.get_enumerators('蘋果','香蕉',n=20)

{ 蘋果: 1.2803411483764648, 香蕉: 1.2803411483764648, 橙子: 1.0296087265014648, 蘋果公司: 0.9841457009315491, 菠蘿: 0.963222086429596, apples: 0.9545719623565674, 梨子: 0.9487223625183105, 橘子: 0.924338698387146, 柚子: 0.9211810827255249, 水梨: 0.9146208167076111, 西紅柿: 0.9081873297691345, applewatch: 0.8966493606567383, 蘋果泥: 0.8853136301040649, 美國蘋果公司: 0.8837926387786865, 火龍果: 0.8834558725357056, 黃瓜: 0.8831906318664551, 庫克: 0.8825984001159668, 黑莓: 0.876366376876831, 小米: 0.8750109672546387, 鳳梨: 0.8745031952857971 }

那要如何排除蘋果作為科技公司的語意呢?簡單，就只要再減掉一個微軟就可以了。

In [22]:
w2v.get_enumerators('蘋果','香蕉',negative_case='微軟',n=20)

{ 香蕉: 1.2249090671539307, 菠蘿: 1.034172773361206, 橙子: 1.0271519422531128, 鳳梨: 0.9487093687057495, 黃瓜: 0.9268301725387573, 火龍果: 0.9219048619270325, 橘子: 0.9158290028572083, 水果: 0.9154829382896423, 草莓: 0.9110264778137207, 柚子: 0.9019840359687805, 西紅柿: 0.8930654525756836, 圣女果: 0.890666663646698, 百香果: 0.8866919279098511, 葡萄干: 0.8833823204040527, 雪糕: 0.8804001808166504, 西瓜: 0.8787985444068909, 梨子: 0.8760404586791992, 櫻桃: 0.8735411167144775, 哈密瓜: 0.8722341656684875, 榴蓮: 0.870747447013855 }

接著我們就可以把剛才的列舉值與類比關係整合再一起，我們看看會得到甚麼??

In [23]:
resuts=w2v.get_enumerators('美國','中國','泰國','德國',n=10,exclude_samples='全球')
for key in resuts.key_list:
    print(key, w2v.analogy(['國家','澳大利亞'], ['首都','坎培拉'], key, n=1))

美國 { 華盛頓: 0.6603407263755798 }
德國 { 慕尼黑: 0.687002420425415 }
中國 { 北京: 0.5113795399665833 }
日本 { 東京: 0.6887738108634949 }
澳大利亞 { 悉尼: 0.6312180757522583 }
印度 { 新德里: 0.6747971773147583 }
法國 { 巴黎: 0.7311732172966003 }
歐洲 { 巴黎: 0.5739215612411499 }
南非 { 開普敦: 0.6316591501235962 }
俄羅斯 { 莫斯科: 0.7113749980926514 }


In [24]:
resuts=w2v.get_enumerators('小米','富士康','格力','百度','企業',negative_case='代工廠',n=10)
for key in resuts.key_list:
    print(key, w2v.analogy('騰訊', '馬化騰', key, n=1))


小米 { 雷軍: 0.5789116621017456 }
格力 { 董明珠: 0.523431658744812 }
華為 { 任正非: 0.5416289567947388 }
百度 { 李彥宏: 0.6010352373123169 }
京東 { 阿里巴巴: 0.4972849488258362 }
阿里巴巴 { 馬云: 0.5887972116470337 }
雷軍 { 柳傳志: 0.5596834421157837 }
富士康 { 郭台銘: 0.4854552149772644 }
阿里 { 馬云: 0.5610132217407227 }
bat { 李彥宏: 0.523595929145813 }


我們也來試試剛才用過星座類比，各位可以觀察一下，使用單純的類比關係跑出來的結果，和我們加上更具體的文字暗示，告訴它我們要找的是性格方面的詞彙，跑出來的結果是有許多的不同。

In [25]:
stars = ['白羊座', '金牛座','雙子座', '巨蟹座', '獅子座', '處女座', '天秤座', '天蝎座', '射手座', '摩羯座', '水瓶座',
             '雙魚座']
for key in stars:
    print(key, w2v.analogy(['雙子座'],['花心'], key, n=5))

白羊座 { 專情: 0.6783907413482666, 專一: 0.6752706170082092, 處女: 0.6722710728645325, 天蝎女: 0.6695566177368164, 大男子主義: 0.6648526191711426 }
金牛座 { 小氣: 0.541993260383606, 專情: 0.5335520505905151, 大男子主義: 0.5258557200431824, 專一: 0.5238892436027527, 厚道: 0.5147523880004883 }
雙子座 { 專情: 0.7229914665222168, 好色: 0.7180087566375732, 處女: 0.6991275548934937, 變心: 0.6933403611183167, 專一: 0.6901572942733765 }
巨蟹座 { 處女: 0.6970769762992859, 專情: 0.6927146315574646, 專一: 0.6858142614364624, 重感情: 0.6763898730278015, 巨蟹: 0.6725313663482666 }
獅子座 { 處女: 0.6827951669692993, 大男子主義: 0.6801401376724243, 天蝎女: 0.6745721101760864, 專情: 0.6715160608291626, 小心眼: 0.6533320546150208 }
處女座 { 處女: 0.7114490866661072, 潔癖: 0.6762194037437439, 專情: 0.6519759297370911, 專一: 0.63627028465271, 好男人: 0.6187808513641357 }
天秤座 { 處女: 0.7107691168785095, 專情: 0.680675208568573, 金牛女: 0.6702103614807129, 專一: 0.6638287901878357, 天蝎女: 0.6528944373130798 }
天蝎座 { 專情: 0.7056789994239807, 處女: 0.7056741714477539, 天蝎: 0.702324628829956, 天蝎女: 0.699754357337951

In [26]:
stars = ['白羊座', '金牛座','雙子座', '巨蟹座', '獅子座', '處女座', '天秤座', '天蝎座', '射手座', '摩羯座', '水瓶座',
             '雙魚座']
for key in stars:
    print(key, w2v.analogy(['雙子座','星座'],['花心','聰明','喜新厭舊','性格'], key, n=5))

白羊座 { 爭強好勝: 0.7158234119415283, 好強: 0.6983242034912109, 不拘小節: 0.691916823387146, 直率: 0.661537766456604, 好勝: 0.6610143184661865 }
金牛座 { 厚道: 0.5623551607131958, 穩重: 0.5453934073448181, 精明: 0.5224258303642273, 沉穩: 0.5054288506507874, 小氣: 0.4996333420276642 }
雙子座 { 善變: 0.6851728558540344, 爭強好勝: 0.6708043813705444, 性情: 0.6469758152961731, 固執: 0.6389719843864441, 不拘小節: 0.6365771293640137 }
巨蟹座 { 重感情: 0.6955631971359253, 溫柔體貼: 0.6886541843414307, 善解人意: 0.684498131275177, 心地善良: 0.6489030122756958, 爭強好勝: 0.6435976028442383 }
獅子座 { 爭強好勝: 0.7128240466117859, 好強: 0.7042689323425293, 懦弱: 0.6535313129425049, 不拘小節: 0.6532725691795349, 好勝: 0.6438640356063843 }
處女座 { 潔癖: 0.6605125069618225, 吹毛求疵: 0.6296414136886597, 偏執: 0.6286723017692566, 完美主義: 0.626579225063324, 不拘小節: 0.6207305788993835 }
天秤座 { 爭強好勝: 0.6361838579177856, 善解人意: 0.6306613087654114, 優柔寡斷: 0.6280789971351624, 隨和: 0.6257694363594055, 不拘小節: 0.6235989928245544 }
天蝎座 { 爭強好勝: 0.6583436131477356, 固執: 0.644228458404541, 懦弱: 0.6392174363136292, 軟